In [68]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from PIL import Image, ImageEnhance, ImageOps
import PIL
import torch.optim as optim
from torch.autograd import Variable
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [69]:
!mkdir best_models
!mkdir checkpoints

mkdir: cannot create directory ‘best_models’: File exists
mkdir: cannot create directory ‘checkpoints’: File exists


In [70]:
# def find_lr(train_loader,criterion_val,init_value = 1e-8, final_value=10., beta = 0.98):
#     num = len(train_loader)-1
#     mult = (final_value / init_value) ** (1/num)
#     lr = init_value
#     optimizer.param_groups[0]['lr'] = lr
#     avg_loss = 0.
#     best_loss = 0.
#     batch_num = 0
#     losses = []
#     log_lrs = []
#     for data in train_loader:
#         batch_num += 1
#         #As before, get the loss for this mini-batch of inputs/outputs
#         inputs,labels = data
#         inputs, labels = Variable(inputs), Variable(labels)
#         optimizer.zero_grad()
#         outputs = student(inputs)
#         loss = criterion_val(outputs, labels)
#         #Compute the smoothed loss
#         avg_loss = beta * avg_loss + (1-beta) *loss.data[0]
#         smoothed_loss = avg_loss / (1 - beta**batch_num)
#         #Stop if the loss is exploding
#         if batch_num > 1 and smoothed_loss > 4 * best_loss:
#             return log_lrs, losses
#         #Record the best loss
#         if smoothed_loss < best_loss or batch_num==1:
#             best_loss = smoothed_loss
#         #Store the values
#         losses.append(smoothed_loss)
#         log_lrs.append(math.log10(lr))
#         #Do the SGD step
#         loss.backward()
#         optimizer.step()
#         #Update the lr for the next step
#         lr *= mult
#         optimizer.param_groups[0]['lr'] = lr
#     return log_lrs, losses


In [71]:
import argparse
import os
import numpy as np
import random
import shutil
import time
import warnings
import random
from enum import Enum
from torch.nn import functional as F
import torch
import torch.backends.cudnn as cudnn
import torch.distributed as dist
import torch.multiprocessing as mp
import torch.nn as nn
import torch.nn.parallel
import torch.optim
import torch.utils.data
import torch.utils.data.distributed
import torchvision.datasets as datasets
import torchvision.models as models
import torchvision.transforms as transforms
from torch.optim.lr_scheduler import StepLR, CosineAnnealingWarmRestarts
from torch.utils.data import Subset
from torch.utils.tensorboard import SummaryWriter
import torchvision
# from cifar_mixup import CIFAR100
# from split_val_test import get_train_valid_loader
from torch.utils.data.sampler import SubsetRandomSampler

import logging
# logging.basicConfig()
# logging.root.setLevel(logging.NOTSET)
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)
logging.debug("test")



best_acc1 = 0


def get_train_valid_loader(data_dir,
                           batch_size,
                           transform_list,
                           random_seed,
                           valid_size=0.1,
                           shuffle=True,
                           show_sample=False,
                           num_workers=4,
                           pin_memory=False):
    """
    Utility function for loading and returning train and valid
    multi-process iterators over the CIFAR-10 dataset. A sample
    9x9 grid of the images can be optionally displayed.
    If using CUDA, num_workers should be set to 1 and pin_memory to True.
    Params
    ------
    - data_dir: path directory to the dataset.
    - batch_size: how many samples per batch to load.
    - augment: whether to apply the data augmentation scheme
      mentioned in the paper. Only applied on the train split.
    - random_seed: fix seed for reproducibility.
    - valid_size: percentage split of the training set used for
      the validation set. Should be a float in the range [0, 1].
    - shuffle: whether to shuffle the train/validation indices.
    - show_sample: plot 9x9 sample grid of the dataset.
    - num_workers: number of subprocesses to use when loading the dataset.
    - pin_memory: whether to copy tensors into CUDA pinned memory. Set it to
      True if using GPU.
    Returns
    -------
    - train_loader: training set iterator.
    - valid_loader: validation set iterator.
    """
    error_msg = "[!] valid_size should be in the range [0, 1]."
    assert ((valid_size >= 0) and (valid_size <= 1)), error_msg

    
    # load the dataset
    test_dataset = torchvision.datasets.CIFAR100(
        root=data_dir, train=False,
        download=True, transform=transform_list,
    )

    valid_dataset = torchvision.datasets.CIFAR100(
        root=data_dir, train=False,
        download=True, transform=transform_list,
    )

    num_test = len(test_dataset)
    indices = list(range(num_test))
    split = int(np.floor(valid_size * num_test))

    if shuffle:
        np.random.seed(random_seed)
        np.random.shuffle(indices)

    train_idx, valid_idx = indices[split:], indices[:split]
    train_sampler = SubsetRandomSampler(train_idx)
    valid_sampler = SubsetRandomSampler(valid_idx)

    test_loader = torch.utils.data.DataLoader(
        test_dataset, batch_size=batch_size, sampler=train_sampler,
        num_workers=num_workers, pin_memory=pin_memory,
    )
    valid_loader = torch.utils.data.DataLoader(
        valid_dataset, batch_size=batch_size, sampler=valid_sampler,
        num_workers=num_workers, pin_memory=pin_memory,
    )
    return test_loader, valid_loader

class EarlyStopper:
    def __init__(self, patience=1, min_delta=0):
        self.patience = patience
        self.min_delta = min_delta
        self.counter = 0
        self.min_validation_loss = np.inf

    def early_stop(self, validation_loss):
        if validation_loss < self.min_validation_loss:
            self.min_validation_loss = validation_loss
            self.counter = 0
        elif validation_loss > (self.min_validation_loss + self.min_delta):
            self.counter += 1
            if self.counter >= self.patience:
                return True
        return False

class CIFAR100Policy(object):
    """ Randomly choose one of the best 25 Sub-policies on CIFAR10.
        Example:
        >>> policy = CIFAR100Policy()
        >>> transformed = policy(image)
        Example as a PyTorch Transform:
        >>> transform=transforms.Compose([
        >>>     transforms.Resize(256),
        >>>     CIFAR10Policy(),
        >>>     transforms.ToTensor()])
    """
    def __init__(self, fillcolor=(128, 128, 128)):
        self.policies = [
            SubPolicy(0.1, "invert", 7, 0.2, "contrast", 6, fillcolor),
            SubPolicy(0.7, "rotate", 2, 0.3, "translateX", 9, fillcolor),
            SubPolicy(0.8, "sharpness", 1, 0.9, "sharpness", 3, fillcolor),
            SubPolicy(0.5, "shearY", 8, 0.7, "translateY", 9, fillcolor),
            SubPolicy(0.5, "autocontrast", 8, 0.9, "equalize", 2, fillcolor),

            SubPolicy(0.2, "shearY", 7, 0.3, "posterize", 7, fillcolor),
            SubPolicy(0.4, "color", 3, 0.6, "brightness", 7, fillcolor),
            SubPolicy(0.3, "sharpness", 9, 0.7, "brightness", 9, fillcolor),
            SubPolicy(0.6, "equalize", 5, 0.5, "equalize", 1, fillcolor),
            SubPolicy(0.6, "contrast", 7, 0.6, "sharpness", 5, fillcolor),

            SubPolicy(0.7, "color", 7, 0.5, "translateX", 8, fillcolor),
            SubPolicy(0.3, "equalize", 7, 0.4, "autocontrast", 8, fillcolor),
            SubPolicy(0.4, "translateY", 3, 0.2, "sharpness", 6, fillcolor),
            SubPolicy(0.9, "brightness", 6, 0.2, "color", 8, fillcolor),
            SubPolicy(0.5, "solarize", 2, 0.0, "invert", 3, fillcolor),

            SubPolicy(0.2, "equalize", 0, 0.6, "autocontrast", 0, fillcolor),
            SubPolicy(0.2, "equalize", 8, 0.8, "equalize", 4, fillcolor),
            SubPolicy(0.9, "color", 9, 0.6, "equalize", 6, fillcolor),
            SubPolicy(0.8, "autocontrast", 4, 0.2, "solarize", 8, fillcolor),
            SubPolicy(0.1, "brightness", 3, 0.7, "color", 0, fillcolor),

            SubPolicy(0.4, "solarize", 5, 0.9, "autocontrast", 3, fillcolor),
            SubPolicy(0.9, "translateY", 9, 0.7, "translateY", 9, fillcolor),
            SubPolicy(0.9, "autocontrast", 2, 0.8, "solarize", 3, fillcolor),
            SubPolicy(0.8, "equalize", 8, 0.1, "invert", 3, fillcolor),
            SubPolicy(0.7, "translateY", 9, 0.9, "autocontrast", 1, fillcolor)
        ]


    def __call__(self, img):
        policy_idx = random.randint(0, len(self.policies) - 1)
        return self.policies[policy_idx](img)

    def __repr__(self):
        return "AutoAugment CIFAR100 Policy"

    
class SubPolicy(object):
    def __init__(self, p1, operation1, magnitude_idx1, p2, operation2, magnitude_idx2, fillcolor=(128, 128, 128)):
        ranges = {
            "shearX": np.linspace(0, 0.3, 10),
            "shearY": np.linspace(0, 0.3, 10),
            "translateX": np.linspace(0, 150 / 331, 10),
            "translateY": np.linspace(0, 150 / 331, 10),
            "rotate": np.linspace(0, 30, 10),
            "color": np.linspace(0.0, 0.9, 10),
            "posterize": np.round(np.linspace(8, 4, 10), 0).astype(np.int),
            "solarize": np.linspace(256, 0, 10),
            "contrast": np.linspace(0.0, 0.9, 10),
            "sharpness": np.linspace(0.0, 0.9, 10),
            "brightness": np.linspace(0.0, 0.9, 10),
            "autocontrast": [0] * 10,
            "equalize": [0] * 10,
            "invert": [0] * 10
        }

        # from https://stackoverflow.com/questions/5252170/specify-image-filling-color-when-rotating-in-python-with-pil-and-setting-expand
        def rotate_with_fill(img, magnitude):
            rot = img.convert("RGBA").rotate(magnitude)
            return Image.composite(rot, Image.new("RGBA", rot.size, (128,) * 4), rot).convert(img.mode)

        func = {
            "shearX": lambda img, magnitude: img.transform(
                img.size, Image.AFFINE, (1, magnitude * random.choice([-1, 1]), 0, 0, 1, 0),
                Image.BICUBIC, fillcolor=fillcolor),
            "shearY": lambda img, magnitude: img.transform(
                img.size, Image.AFFINE, (1, 0, 0, magnitude * random.choice([-1, 1]), 1, 0),
                Image.BICUBIC, fillcolor=fillcolor),
            "translateX": lambda img, magnitude: img.transform(
                img.size, Image.AFFINE, (1, 0, magnitude * img.size[0] * random.choice([-1, 1]), 0, 1, 0),
                fillcolor=fillcolor),
            "translateY": lambda img, magnitude: img.transform(
                img.size, Image.AFFINE, (1, 0, 0, 0, 1, magnitude * img.size[1] * random.choice([-1, 1])),
                fillcolor=fillcolor),
            "rotate": lambda img, magnitude: rotate_with_fill(img, magnitude),
            # "rotate": lambda img, magnitude: img.rotate(magnitude * random.choice([-1, 1])),
            "color": lambda img, magnitude: ImageEnhance.Color(img).enhance(1 + magnitude * random.choice([-1, 1])),
            "posterize": lambda img, magnitude: ImageOps.posterize(img, magnitude),
            "solarize": lambda img, magnitude: ImageOps.solarize(img, magnitude),
            "contrast": lambda img, magnitude: ImageEnhance.Contrast(img).enhance(
                1 + magnitude * random.choice([-1, 1])),
            "sharpness": lambda img, magnitude: ImageEnhance.Sharpness(img).enhance(
                1 + magnitude * random.choice([-1, 1])),
            "brightness": lambda img, magnitude: ImageEnhance.Brightness(img).enhance(
                1 + magnitude * random.choice([-1, 1])),
            "autocontrast": lambda img, magnitude: ImageOps.autocontrast(img),
            "equalize": lambda img, magnitude: ImageOps.equalize(img),
            "invert": lambda img, magnitude: ImageOps.invert(img)
        }

        # self.name = "{}_{:.2f}_and_{}_{:.2f}".format(
        #     operation1, ranges[operation1][magnitude_idx1],
        #     operation2, ranges[operation2][magnitude_idx2])
        self.p1 = p1
        self.operation1 = func[operation1]
        self.magnitude1 = ranges[operation1][magnitude_idx1]
        self.p2 = p2
        self.operation2 = func[operation2]
        self.magnitude2 = ranges[operation2][magnitude_idx2]


    def __call__(self, img):
        if random.random() < self.p1: img = self.operation1(img, self.magnitude1)
        if random.random() < self.p2: img = self.operation2(img, self.magnitude2)
        return img
    
    
def main(args):
    global best_acc1
    global device

    writer = SummaryWriter(log_dir=os.path.join("tensorboard", args.experiment_name))

    if torch.cuda.is_available():
        device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
        print("Use GPU for training : CUDA version {}, device:{}".format(torch.version.cuda,device))

    # create model
#     teacher=args.teacher
    student=args.student
    print("=> Loading Student model '{}'".format(student._get_name()))
#     logging.info("=> Loading Teacher model '{}'".format(teacher._get_name()))
        

    if not torch.cuda.is_available():
        print('using CPU, this will be slow')

#     teacher = teacher.to(device)
    student = student.to(device)

    # define loss function (criterion), optimizer, and learning rate scheduler
    criterion_val = nn.CrossEntropyLoss().to(device)
#     criterion=torch.nn.KLDivLoss(reduction="sum").to(device)

    optimizer = torch.optim.Adam(student.parameters(), lr=args.lr,
                                weight_decay=args.weight_decay)
    
    """Sets the learning rate to the initial LR decayed by 10 every 30 epochs"""
    """scheduler = StepLR(optimizer, step_size=250, gamma=0.1)"""
    scheduler = CosineAnnealingWarmRestarts(optimizer, T_0 = args.epochs + 1, T_mult=1, eta_min=args.min_lr, verbose=False)
    
#     scheduler = CosineAnnealingWarmRestarts(optimizer, T_0 = 30, T_mult=1, eta_min=args.min_lr, verbose=False)
    
    # optionally resume from a checkpoint
    if args.resume:
        
        if os.path.isfile(args.resume):
            print("=> loading checkpoint '{}'".format(args.resume))
            if torch.cuda.is_available():
                checkpoint = torch.load(args.resume)
            args.start_epoch = checkpoint['epoch']
            best_acc1 = checkpoint['best_acc1']

            student.load_state_dict(checkpoint['state_dict'])
            optimizer.load_state_dict(checkpoint['optimizer'])
            scheduler.load_state_dict(checkpoint['scheduler'])
            print("=> loaded checkpoint '{}' (epoch {})"
                  .format(args.resume, checkpoint['epoch']))
        else:
            print("=> no checkpoint found at '{}'".format(args.resume))


#     train_transform = transforms.Compose(
#     [transforms.RandomHorizontalFlip(),
#         transforms.RandomRotation(20),transforms.ToTensor(),
#      transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
    
    train_transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
    
    val_transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])


   
    train_dataset=torchvision.datasets.CIFAR100('./data', train=True, download=True,transform=train_transform)
    
    test_loader, val_loader = get_train_valid_loader("./data",
                            batch_size = 4,
                            transform_list=val_transform,
                            random_seed = 1,
                            valid_size=0.5,
                            shuffle=True,
                            show_sample=False,
                            num_workers=1,
                            pin_memory=True)

    train_loader = torch.utils.data.DataLoader(
        train_dataset, batch_size=args.batch_size, shuffle=True,
        num_workers=args.workers)


    """ if args.evaluate:
        validate(val_loader, student, criterion, args)
        return"""
    early_stopper = EarlyStopper(patience=3, min_delta=10)
    
#     logs,losses = find_lr(train_loader,criterion_val)
#     plt.plot(logs[10:-5],losses[10:-5])
    for epoch in range(args.start_epoch, args.epochs):

        # train for one epoch
        writer = train(train_loader, student, criterion_val, optimizer, epoch, device, writer, args)

        # evaluate on validation set # change this as this is the measure of whether the model is good or not versus the training/ validation set
        acc1, writer = validate(val_loader, student, criterion_val, writer, epoch, args)
        
        scheduler.step()
        
        # remember best acc@1 and save checkpoint
        is_best = acc1 > best_acc1
        best_acc1 = max(acc1, best_acc1)

        if early_stopper.early_stop(acc1):             
            break
        save_checkpoint(
                state={'epoch': epoch + 1,
                'student': args.student,
                'state_dict': student.state_dict(),
                'best_acc1': best_acc1,
                'optimizer' : optimizer.state_dict(),
                'scheduler' : scheduler.state_dict()
                }, 
                is_best=is_best,
                args=args,
                filename=os.path.join("checkpoints",args.experiment_name +'_checkpoint.pth.tar'))
    writer.flush()

def train(train_loader, student, criterion, optimizer, epoch, device, writer, args):
    batch_time = AverageMeter('Time', ':6.3f')
    # data_time = AverageMeter('Data', ':6.3f')
    losses = AverageMeter('Loss', ':.4e')
    top1 = AverageMeter('Acc@1', ':6.2f')
    top5 = AverageMeter('Acc@5', ':6.2f')

#     top1_teacher = AverageMeter('Teacher Acc@1', ':6.2f')
#     top5_teacher = AverageMeter('Teacher Acc@5', ':6.2f')
    progress = ProgressMeter(
        len(train_loader),
        [batch_time, losses, top1, top5],
        prefix="Epoch: [{}]".format(epoch))

    # switch to train mode
    student.train()
#     teacher.eval()

    end = time.time()
    for i, (images, target) in enumerate(train_loader):
        # measure data loading time
        # data_time.update(time.time() - end)

        # move data to the same device as model
        images = images.to(device)#, non_blocking=True)
        target = target.to(device)#, non_blocking=True)
        # writer.add_graph(student, images)
        # writer.add_graph(teacher, images)
        optimizer.zero_grad()
        
        # compute output
        output_student_raw = student(images)
#         output_teacher_raw = teacher(images)



#         output_student = F.log_softmax(output_student_raw/args.temperature, dim=1)
#         output_teacher = F.softmax(output_teacher_raw/args.temperature, dim=1)
        loss = criterion(output_student_raw, target)

        # measure accuracy and record loss
        acc1, acc5 = accuracy(output_student_raw, target, topk=(1, 5))
        #for the teacher
#         t_acc1, t_acc5 = accuracy(output_teacher_raw, target, topk=(1, 5))

        losses.update(loss.item(), images.size(0))
        top1.update(acc1[0], images.size(0))
        top5.update(acc5[0], images.size(0))

#         top1_teacher.update(t_acc1[0], images.size(0))
#         top5_teacher.update(t_acc5[0], images.size(0))
        # compute gradient and do SGD step
        
        loss.backward()
        optimizer.step()
        writer.add_scalar("Loss/train", loss, epoch)
        writer.add_scalar("Accuracy_student/train", acc1, epoch)
#         writer.add_scalar("Accuracy_teacher/train", t_acc1, epoch)
        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        if i % args.print_freq == 0:
            progress.display(i + 1)
    return writer
    

def validate(val_loader, student, criterion, writer, epoch, args):

    def run_validate(loader, base_progress=0):
        with torch.no_grad():
            end = time.time()
            for i, (images, target) in enumerate(loader):
                i = base_progress + i
                images = images.to(device, non_blocking=True)
                target = target.to(device, non_blocking=True)

                # compute output
                output = student(images)
                loss = criterion(output, target)

                # measure accuracy and record loss
                acc1, acc5 = accuracy(output, target, topk=(1, 5))
                losses.update(loss.item(), images.size(0))
                top1.update(acc1[0], images.size(0))
                top5.update(acc5[0], images.size(0))

                # measure elapsed time
                batch_time.update(time.time() - end)
                end = time.time()

                if i % args.print_freq == 0:
                    progress.display(i + 1)

    batch_time = AverageMeter('Time', ':6.3f', Summary.NONE)
    losses = AverageMeter('Loss', ':.4e', Summary.NONE)
    top1 = AverageMeter('Acc@1', ':6.2f', Summary.AVERAGE)
    top5 = AverageMeter('Acc@5', ':6.2f', Summary.AVERAGE)
    progress = ProgressMeter(
        len(val_loader) + ((len(val_loader.sampler) < len(val_loader.dataset))),
        [batch_time, losses, top1, top5],
        prefix='Test: ')

    # switch to evaluate mode
    student.eval()

    run_validate(val_loader)
    # if args.distributed:
    #     top1.all_reduce()
    #     top5.all_reduce()

    if (len(val_loader.sampler) < len(val_loader.dataset)):
        aux_val_dataset = Subset(val_loader.dataset,
                                 range(len(val_loader.sampler), len(val_loader.dataset)))
        aux_val_loader = torch.utils.data.DataLoader(
            aux_val_dataset, batch_size=args.batch_size, shuffle=False,
            num_workers=args.workers, pin_memory=True)
        run_validate(aux_val_loader, len(val_loader))

    progress.display_summary()
    
    writer.add_scalar("Loss_student/Val", losses.avg, epoch)
    writer.add_scalar("Accuracy_student/Val", top1.avg, epoch)

    return top1.avg, writer


def save_checkpoint(state, is_best, args,filename='checkpoint.pth.tar'):
    torch.save(state, filename)
    if is_best:
        shutil.copyfile(filename, os.path.join('best_models',args.experiment_name+'_model_best.pth.tar'))

class Summary(Enum):
    NONE = 0
    AVERAGE = 1
    SUM = 2
    COUNT = 3

class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self, name, fmt=':f', summary_type=Summary.AVERAGE):
        self.name = name
        self.fmt = fmt
        self.summary_type = summary_type
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

    def all_reduce(self):
        if torch.cuda.is_available():
            device = torch.device("cuda")
        else:
            device = torch.device("cpu")
        total = torch.tensor([self.sum, self.count], dtype=torch.float32, device=device)
        # dist.all_reduce(total, dist.ReduceOp.SUM, async_op=False)
        self.sum, self.count = total.tolist()
        self.avg = self.sum / self.count

    def __str__(self):
        fmtstr = '{name} {val' + self.fmt + '} ({avg' + self.fmt + '})'
        return fmtstr.format(**self.__dict__)
    
    def summary(self):
        fmtstr = ''
        if self.summary_type is Summary.NONE:
            fmtstr = ''
        elif self.summary_type is Summary.AVERAGE:
            fmtstr = '{name} {avg:.3f}'
        elif self.summary_type is Summary.SUM:
            fmtstr = '{name} {sum:.3f}'
        elif self.summary_type is Summary.COUNT:
            fmtstr = '{name} {count:.3f}'
        else:
            raise ValueError('invalid summary type %r' % self.summary_type)
        
        return fmtstr.format(**self.__dict__)


class ProgressMeter(object):
    def __init__(self, num_batches, meters, prefix=""):
        self.batch_fmtstr = self._get_batch_fmtstr(num_batches)
        self.meters = meters
        self.prefix = prefix

    def display(self, batch):
        entries = [self.prefix + self.batch_fmtstr.format(batch)]
        entries += [str(meter) for meter in self.meters]
        print('\t'.join(entries))
        
    def display_summary(self):
        entries = [" *"]
        entries += [meter.summary() for meter in self.meters]
        print(' '.join(entries))

    def _get_batch_fmtstr(self, num_batches):
        num_digits = len(str(num_batches // 1))
        fmt = '{:' + str(num_digits) + 'd}'
        return '[' + fmt + '/' + fmt.format(num_batches) + ']'

def accuracy(output, target, topk=(1,)):
    """Computes the accuracy over the k top predictions for the specified values of k"""
    with torch.no_grad():
        maxk = max(topk)
        batch_size = target.size(0)

        _, pred = output.topk(maxk, 1, True, True)
        pred = pred.t()
        correct = pred.eq(target.view(1, -1).expand_as(pred))

        res = []
        for k in topk:
            correct_k = correct[:k].reshape(-1).float().sum(0, keepdim=True)
            res.append(correct_k.mul_(100.0 / batch_size))
        return res
def add_pr_curve_tensorboard(class_index, test_probs, test_label,classes, global_step=0):
    '''
    Takes in a "class_index" from 0 to 9 and plots the corresponding
    precision-recall curve
    '''
    tensorboard_truth = test_label == class_index
    tensorboard_probs = test_probs[:, class_index]

    writer.add_pr_curve(classes[class_index],
                        tensorboard_truth,
                        tensorboard_probs,
                        global_step=global_step)
    writer.close()

In [72]:
# from main_training import main
from torchvision.models import mobilenet_v3_small
student = mobilenet_v3_small(pretrained=False, num_classes=100)


In [73]:
import torch
# from main_training import main
from torchvision.models import mobilenet_v3_small
from torch.optim.lr_scheduler import StepLR, CosineAnnealingWarmRestarts
import matplotlib.pyplot as plt

class argclass(object):
    def __init__(self, *initial_data,**kwargs):
        for dictionary in initial_data:
            for key in dictionary:
                setattr(self, key, dictionary[key])
        for key in kwargs:
            setattr(self, key, kwargs[key])

# teacher3=torch.hub.load("chenyaofo/pytorch-cifar-models",'cifar100_mobilenetv2_x1_0',pretrained=True)

hyperparams={'lr':0.001,
              'min_lr':0,
             'momentum':0.9,
             'weight_decay':1e-4,
             'batch_size':128,
             'epochs':25,
             'start_epoch':0,
             'print_freq':50,
             'pretrained':True,
             'evaluate':False,
             'world_size':-1,
             'rank':-1,
             'dist_url':'tcp://',
             'dist_backend':'nccl',
             'seed':None,
             'gpu':None,
             'multiprocessing_distributed':False,
             'dummy':False,
             'data':'./data',
             'workers':4,
            #  'resume':'model_best.pth.tar',
             'resume':'',
             'evaluate':False,
#              'teacher':teacher3,
             'student':mobilenet_v3_small(pretrained=False, num_classes=100),
             "experiment_name":"test-student_from_scratch-v1",
#              'temperature':5
            }

args=argclass(hyperparams)


In [74]:
main(args)

Use GPU for training : CUDA version 11.8, device:cuda:0
=> Loading Student model 'MobileNetV3'
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Epoch: [0][  1/391]	Time  0.478 ( 0.478)	Loss 4.6026e+00 (4.6026e+00)	Acc@1   1.56 (  1.56)	Acc@5   6.25 (  6.25)
Epoch: [0][ 51/391]	Time  0.040 ( 0.054)	Loss 4.5151e+00 (4.4991e+00)	Acc@1   0.78 (  2.47)	Acc@5  14.06 ( 10.68)
Epoch: [0][101/391]	Time  0.039 ( 0.050)	Loss 4.1407e+00 (4.3568e+00)	Acc@1   3.91 (  3.49)	Acc@5  20.31 ( 14.74)
Epoch: [0][151/391]	Time  0.065 ( 0.050)	Loss 4.0699e+00 (4.2576e+00)	Acc@1   3.91 (  4.55)	Acc@5  24.22 ( 17.62)
Epoch: [0][201/391]	Time  0.041 ( 0.049)	Loss 3.9177e+00 (4.1930e+00)	Acc@1   8.59 (  5.21)	Acc@5  28.91 ( 19.59)
Epoch: [0][251/391]	Time  0.038 ( 0.048)	Loss 3.7243e+00 (4.1393e+00)	Acc@1  10.16 (  5.84)	Acc@5  31.25 ( 21.18)
Epoch: [0][301/391]	Time  0.040 ( 0.047)	Loss 3.9762e+00 (4.0922e+00)	Acc@1   3.91 (  6.46)	Acc@5  25.78 ( 